<a href="https://colab.research.google.com/github/NimdaGrogu/Machine_Learning_Projects/blob/dev/Generative%20AI%20Foundations%20/Capstone%20Projects/Project%202%20Medical%20Assistant/Learners_Notebook_Full_Code_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

1. **Critical Care Protocols:** "What is the protocol for managing sepsis in a critical care unit?"

2. **General Surgery:** "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

3. **Dermatology:** "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

4. **Neurology:** "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"


### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [2]:
!pip install --upgrade pip -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.8 MB/s eta 0:00:00


In [3]:
# Install required libraries
!pip install -q langchain_community==0.3.27 \
              langchain==0.3.27 \
              chromadb==1.0.15 \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5 \
              langchain_openai==0.3.30 \
              langchain-chroma \
              langchain-google-genai \
              langchain_core




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetr

In [4]:
# Import core libraries
import os                                                                       # Interact with the operating system (e.g., set environment variables)
import json                                                                     # Read/write JSON data
import requests  # type: ignore                                                 # Make HTTP requests (e.g., API calls); ignore type checker

# Import libraries for working with PDFs and OpenAI
from langchain.document_loaders import PyMuPDFLoader                            # Load and extract text from PDF files
# from langchain_community.document_loaders import PyPDFLoader                    # Load and extract text from PDF files
from openai import OpenAI                                                       # Access OpenAI's models and services


# Import libraries for Gemini
from langchain_google_genai.llms import ChatGoogleGenerativeAI

# Import libraries for processing dataframes and text
import tiktoken                                                                 # Tokenizer used for counting and splitting text for models
import pandas as pd                                                             # Load, manipulate, and analyze tabular data

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain.text_splitter import RecursiveCharacterTextSplitter              # Break text into overlapping chunks for processing
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings                                   # Create vector embeddings using OpenAI's models  # type: ignore
#from langchain.vectorstores import Chroma                                      # Store and search vector embeddings using Chroma DB  # type: ignore
from langchain_chroma import Chroma                                             # Store and search vector embeddings using Chroma DB  # type: ignore
from datasets import Dataset                                                    # Used to structure the input (questions, answers, contexts etc.) in tabular format
from langchain_openai import ChatOpenAI                                         # This is needed since LLM is used in metric computation

from langchain_google_genai import ChatGoogleGenerativeAI                       # Google Gemini
from langchain.memory import ConversationBufferMemory

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

## Loading OpenAI configuration

In [5]:
import json # Import the json module

# Load the JSON file and extract values
file_name = "Config.json"                                                       # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")                                             # Extract the API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")
    GOOGLE_API_KEY = config.get("GOOGLE_API_KEY")                                             # Extract the API key from the config
                              # Extract the OpenAI base URL from the config

# Store API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY                                          # Set API key as environment variable
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE                                 # Set API base URL as environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize OpenAI client
client = OpenAI()

## Question Answering using LLM

## Question Answering with OpenAI



In [6]:
#Define a decorator functin
import textwrap, re
def clean_output(func=None):
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)

    # Check if the result from the function is a string before attempting to clean
    if isinstance(result, str):
      output = ' '.join(result)
      output = textwrap.fill(output, width=100)

      return output
    return result
  return wrapper

# Define a function to get a response
# @clean_output
def response(user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):   # Complete the code to set default paramenters
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",  # Complete the code by specifying the model to be used.
        messages=[
            {"role": "user",
             "content": user_prompt
             }                            # User prompt is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [7]:
q1 = "What is the protocol for managing sepsis in a critical care unit?"

base_prompt_response_1 = response(q1)
print(base_prompt_response_1)


The management of sepsis in a critical care unit is a high-priority and complex process, guided by evolving evidence-based practices. As of my last update in 2023, the following steps provide a general outline of the protocol typically used in managing sepsis in critical care settings. These steps are based on guidelines such as those from the Surviving Sepsis Campaign (SSC) and other clinical resources:

1. **Early Identification and Diagnosis**:
   - Recognize symptoms and signs of sepsis early. Common symptoms include fever, hypothermia, tachycardia, tachypnea, confusion, edema, or hyperglycemia without other cause.
   - Use screening tools and laboratory tests to aid in diagnosis. Important tests include blood cultures, complete blood count, serum lactate levels, and organ function tests.

2. **Immediate Management**:
   - **Initial Resuscitation**: Follow the SSC guidelines for sepsis which recommend the initiation of treatment within the first hour of recognition of sepsis. This 

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [8]:

q2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

base_prompt_response_2 = response(user_prompt=q2, temperature=0.5, top_p=0.9)
print(base_prompt_response_2)


Appendicitis is an inflammation of the appendix, a small, finger-shaped pouch attached to the large intestine on the lower right side of the abdomen. It is a common condition and a medical emergency that typically requires prompt treatment.

### Common Symptoms of Appendicitis

1. **Abdominal Pain**: The most notable symptom is pain that begins near the navel and then moves to the lower right abdomen. The pain typically worsens over a period of 12 to 24 hours and becomes severe.
2. **Appetite Loss**
3. **Nausea and Vomiting**
4. **Abdominal Swelling**
5. **Fever**: Usually low-grade, near 99-102 degrees Fahrenheit.
6. **Inability to Pass Gas**
7. **Constipation or Diarrhea**: These symptoms can occur but are less common.
8. **Painful Urination**: Occasionally observed if the inflamed appendix is near the bladder or ureter.

### Treatment for Appendicitis

Appendicitis typically requires surgical intervention. Medications alone cannot cure appendicitis, especially if the appendix has ru

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [9]:
q3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

base_prompt_response_3 = response(user_prompt=q3, temperature=0.5, top_p=0.9)
print(base_prompt_response_3)

Sudden patchy hair loss, which results in localized bald spots on the scalp, is most commonly associated with a condition called alopecia areata. Alopecia areata is an autoimmune disorder in which the immune system attacks the hair follicles, leading to hair loss. The hair loss can be just a few spots or more extensive, and it can also affect other parts of the body.

### Possible Causes of Alopecia Areata:
1. **Autoimmune Response**: The exact cause is unknown, but it involves the immune system attacking the hair follicles.
2. **Genetic Factors**: There is often a family history of alopecia areata or other autoimmune conditions.
3. **Stress**: Emotional or physical stress has been implicated as a trigger for some people, though the link is not fully understood.
4. **Hormonal Changes**: Changes in hormones may also play a role, as suggested by the onset in some women post-pregnancy or at menopause.
5. **Environmental Factors**: Some studies suggest that environmental factors might trig

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [10]:
q4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"

base_prompt_response_4 = response(user_prompt=q4, temperature=0.7, top_p=0.9)
print(base_prompt_response_4)

Treating a physical injury to brain tissue, commonly known as traumatic brain injury (TBI), involves multiple steps and disciplines to address both immediate and long-term effects. The treatment approach depends on the severity of the injury, which can be classified generally as mild, moderate, or severe. Here are the recommended treatments for each stage and aspect of care:

### Immediate Medical Care
1. **Emergency Treatment**: Immediate goals include ensuring the patient's airway, breathing, and circulation are adequate (the ABCs), controlling external bleeding, and preventing further injury to the brain and other parts of the body.
2. **Medical Imaging**: CT scans are typically used to diagnose brain injuries and determine the extent and location of the damage.
3. **Surgery**: This may be necessary to address issues such as bleeding in the brain (hematomas), repairing skull fractures, or relieving elevated intracranial pressure.

### Hospital Treatment
1. **Medications**:
   - **Di

### Storing the responds

In [11]:
responses = pd.DataFrame({
    "Question": [q1, q2, q3, q4],
    "Base Prompt Response": [base_prompt_response_1, base_prompt_response_2, base_prompt_response_3, base_prompt_response_4]
    }
)

## Question Answering using LLM with Prompt Engineering

### Defining the function to Generate a Response From the LLM with Prompt Engineering


In [12]:
# Define a function to get a response from the OpenAI chat model
def response_with_prompt_eng(system_prompt, user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):  # Complete the code to set default paramenters
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",                                                        # Complete the code by specifying the model to be used.
        messages=[
            {"role": "system", "content": system_prompt},                       # System prompt sets the assistant's behavior
            {"role": "user", "content": user_prompt}                            # User prompt is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output (0 = deterministic)
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

#### Define the prompts for system and user

In [13]:
system_prompt = ("""
    You are an AI medical assistant specialized in providing comprehensive medical information based on the Merck Manuals.
    Your primary goal is to offer accurate, concise, and helpful information for general understanding only.
    You must strictly adhere to the following guidelines:
    - Provide answers based solely on general medical knowledge.
    - Do NOT provide personalized medical advice, diagnoses, or treatment recommendations for individuals.
    - Always maintain a professional, informative, and neutral tone.
    - Clarify that your responses are for informational purposes only and not a substitute for professional medical consultation.
    - Focus on explaining conditions, symptoms, and general treatment approaches rather than prescribing actions.
    """)



### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [14]:
response_with_prompt_eng_1 = response_with_prompt_eng(system_prompt,q1)
print(response_with_prompt_eng_1)


Sepsis is a life-threatening condition triggered by an infection that leads to an overwhelming immune response, potentially resulting in organ failure and death. Managing sepsis in a critical care unit is a high-priority and complex process that involves several key steps aimed at stabilizing the patient and combating the infection. The protocol generally follows these guidelines:

1. **Early Identification and Diagnosis**: Rapid recognition of sepsis is crucial. This involves monitoring for signs of sepsis in patients, especially those who are already critically ill or have a high risk due to recent surgery, invasive procedures, or weakened immune systems. Common signs include fever, increased heart rate, increased respiratory rate, confusion, and other changes in mental status.

2. **Immediate Treatment**: The cornerstone of sepsis treatment is the administration of broad-spectrum antibiotics as soon as sepsis is suspected or diagnosed. This is often done before the specific cause of

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
response_with_prompt_eng_2=response_with_prompt_eng(system_prompt,q2)
print(response_with_prompt_eng_2)

Appendicitis is an inflammation of the appendix, a small organ attached to the large intestine. Common symptoms of appendicitis include:

1. **Abdominal pain**: This pain usually starts around the navel and then moves to the lower right side of the abdomen. The pain typically worsens with movement, deep breathing, coughing, or sneezing.
2. **Loss of appetite**
3. **Nausea and vomiting**
4. **Fever**: Usually not very high.
5. **Abdominal swelling**

Appendicitis is a medical emergency, and it is important to seek immediate medical attention if appendicitis is suspected. The condition typically requires surgical intervention and is not effectively treated with medication alone.

The standard treatment for appendicitis is surgical removal of the appendix, a procedure known as an appendectomy. There are two main types of appendectomy:

1. **Laparoscopic appendectomy**: This is a minimally invasive surgery where small incisions are made, and a camera (laparoscope) and surgical tools are in

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
response_with_prompt_eng_3=response_with_prompt_eng(system_prompt,q3)
print(response_with_prompt_eng_3)


Sudden patchy hair loss, which typically appears as localized bald spots on the scalp, is often due to a condition known as alopecia areata. This is an autoimmune disorder where the immune system mistakenly attacks the hair follicles, leading to hair loss.

**Possible Causes:**
- **Autoimmune response:** The primary factor in alopecia areata, where the body's immune system targets healthy hair follicles.
- **Genetic predisposition:** There may be a genetic component, as the condition sometimes runs in families.
- **Stress:** Psychological stress has been suggested as a possible trigger for the onset or exacerbation of the condition, though the exact relationship is not fully understood.

**Effective Treatments:**
1. **Corticosteroids:** These are powerful anti-inflammatory drugs that can suppress the immune system. They can be administered through local injections into the scalp, topical application (creams and ointments), or orally.
2. **Minoxidil (Rogaine):** Originally used for trea

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [17]:
response_with_prompt_eng_4=response_with_prompt_eng(system_prompt,q4)
print(response_with_prompt_eng_4)

Treatment for brain injuries, particularly those that result in temporary or permanent impairment of brain function, depends on the severity and type of the injury. Here is a general overview of the approaches typically recommended:

1. **Immediate Medical Care**: The first step in treating a brain injury is to stabilize the individual. This includes ensuring that the airway is clear, breathing and circulation are adequate, and preventing further injury to the head and neck.

2. **Medication**: Depending on the symptoms and severity, medications may be used to control symptoms such as seizures, pain, and to manage other complications that can arise after a brain injury.

3. **Surgery**: In cases where there are blood clots (hematomas), bruising of the brain tissue (contusions), or increased pressure inside the skull, surgical intervention might be necessary to relieve pressure and prevent further damage.

4. **Rehabilitation**: After the initial treatment, rehabilitation is crucial for

#### Storing the generated output

In [18]:
responses["Response with Prompt Engineering"] = [response_with_prompt_eng_1, response_with_prompt_eng_2, response_with_prompt_eng_3, response_with_prompt_eng_4]
responses

,Question,Base Prompt Response,Response with Prompt Engineering
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a life-threatening condition trigger...
1,"What are the common symptoms for appendicitis,...",Appendicitis is an inflammation of the appendi...,Appendicitis is an inflammation of the appendi...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, which typically appea..."
3,What treatments are recommended for a person w...,"Treating a physical injury to brain tissue, co...","Treatment for brain injuries, particularly tho..."


## Data Preparation for RAG

### Loading the Data

In [19]:
manual_pdf = "medical_diagnosis_manual.pdf"
pdf_loader = PyMuPDFLoader(manual_pdf)
data = pdf_loader.load()

### Data Overview

In [20]:

for i in range(5):
  print(f"Page Number: {i+1}", end="\n")
  print(data[i].page_content,end="\n")


Page Number: 1
jonathan@nimda.sh
4J6EVYSP7A
meant for personal use by jonathan@nim
shing the contents in part or full is liable
Page Number: 2
jonathan@nimda.sh
4J6EVYSP7A
This file is meant for personal use by jonathan@nimda.sh only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number: 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nutrition

### Data Chunking
#### Chunk the PDF into Manageable Text Sections Using a Token-Based Splitter


In [21]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=512,
    chunk_overlap=200,
)

In [22]:
document_chunks = text_splitter.split_documents(data)
#document_chunks = pdf_loader.load_and_split(text_splitter)


In [23]:
len(document_chunks)

10115

### Embedding



### Embeddings from OpenAI

In [24]:
# Initialize the OpenAI Embeddings model with API credentials
embedding_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,                                                     # Your OpenAI API key for authentication
    openai_api_base=OPENAI_API_BASE                                             # The OpenAI API base URL endpoint
)

# Generate embeddings (vector representations) for the first two document chunks
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)      # Embedding for chunk 0
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)      # Embedding for chunk 1

# Check and print the dimension (length) of the embedding vector
print("Dimension of the embedding vector ", len(embedding_1))                   # Typically 1536 or 2048 depending on model

Dimension of the embedding vector  1536


In [25]:
# Verify if both embeddings have the same dimension (should be True)
len(embedding_1) == len(embedding_2)


True

In [26]:

# Return/display the two embedding vectors for further inspection or use
embedding_1, embedding_2

([-0.006342974025756121,
  0.006594567093998194,
  -0.010092061944305897,
  -0.009766054339706898,
  -0.02007072977721691,
  0.02442222274839878,
  -0.01943288929760456,
  -0.020198296755552292,
  -0.02864614687860012,
  -0.0038872864097356796,
  0.029907654970884323,
  0.021204669028520584,
  0.01656969077885151,
  0.0039652446284890175,
  0.011013387702405453,
  0.009121125563979149,
  0.018383994698524475,
  -0.015563319437205791,
  0.012912736274302006,
  -0.0006940949824638665,
  -0.010538550093770027,
  0.008164364844560623,
  -0.014117547310888767,
  0.017505191266536713,
  -0.013380486518144608,
  -0.020042380318045616,
  0.003153769299387932,
  -0.031240032985806465,
  -0.00026266646455042064,
  -0.01919192634522915,
  0.027342116460204124,
  -0.015138092450797558,
  -0.00014894029300194234,
  -0.02167241834104061,
  -0.02673262357711792,
  -0.0016043472569435835,
  0.00439401576295495,
  -0.025017540901899338,
  0.025669556111097336,
  -0.001118879416026175,
  -0.005850419402

### Vector Database



#### Setup Vector Database Directory

In [27]:
out_dir = 'vector_db'    # name of the vector database

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [28]:
(len(document_chunks) +1) // (300 +1)


33

In [29]:
# Initialize an empty Chroma vector store for persistence
vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory=out_dir
)

# Define a batch size for adding documents to avoid hitting API token limits
batch_size = 300

# Add documents in batches
for i in range(0, len(document_chunks), batch_size):
    batch = document_chunks[i : i + batch_size]
    print(f"Adding batch {i // batch_size + 1}/{(len(document_chunks) -1) // batch_size + 1} with {len(batch)} documents...")
    vectorstore.add_documents(documents=batch)

print("Vector store created and saved.")

Adding batch 1/34 with 300 documents...
Adding batch 2/34 with 300 documents...
Adding batch 3/34 with 300 documents...
Adding batch 4/34 with 300 documents...
Adding batch 5/34 with 300 documents...
Adding batch 6/34 with 300 documents...
Adding batch 7/34 with 300 documents...
Adding batch 8/34 with 300 documents...
Adding batch 9/34 with 300 documents...
Adding batch 10/34 with 300 documents...
Adding batch 11/34 with 300 documents...
Adding batch 12/34 with 300 documents...
Adding batch 13/34 with 300 documents...
Adding batch 14/34 with 300 documents...
Adding batch 15/34 with 300 documents...
Adding batch 16/34 with 300 documents...
Adding batch 17/34 with 300 documents...
Adding batch 18/34 with 300 documents...
Adding batch 19/34 with 300 documents...
Adding batch 20/34 with 300 documents...
Adding batch 21/34 with 300 documents...
Adding batch 22/34 with 300 documents...
Adding batch 23/34 with 300 documents...
Adding batch 24/34 with 300 documents...
Adding batch 25/34 with 3

#### Load Vector DataDB

In [30]:
vectorstore = Chroma(
    persist_directory=out_dir,
    embedding_function=embedding_model
)

In [31]:
vectorstore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7a3bf74fa0c0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7a3bf74fb5f0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [32]:
vectorstore.similarity_search(q1, k=3)

[Document(id='561ae337-3095-48f7-9cb0-f49578d4251e', metadata={'creationDate': 'D:20120615054440Z', 'format': 'PDF 1.7', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': 'medical_diagnosis_manual.pdf', 'total_pages': 4114, 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'modDate': 'D:20251107191215Z', 'keywords': '', 'page': 2456, 'moddate': '2025-11-07T19:12:15+00:00', 'author': '', 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'file_path': 'medical_diagnosis_manual.pdf', 'creator': 'Atop CHM to PDF Converter', 'subject': '', 'trapped': ''}, page_content="Parenteral antibiotics should be given after specimens of blood, body fluids, and wound sites have been\ntaken for Gram stain and culture. Very prompt empiric therapy, started immediately after suspecting\nsepsis, is essential and may be lifesaving. Antibiotic selection requires an educated guess based on the\nsuspected source, clinical setting, knowledge or suspicion of causative organisms and o

#### Convert Vector Database into a Retriever and Retrieve Relevant Documents


### Retriever

In [33]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

### RAG System and User Prompt Template

In [34]:
rag_system_prompt = ("""
    You are an AI medical assistant specialized in providing comprehensive medical information based on the Merck Manuals.
    Your primary goal is to offer accurate, concise, and helpful information for general understanding only.
    You must strictly adhere to the following guidelines:
    - Provide answers based solely on general medical knowledge.
    - Do NOT provide personalized medical advice, diagnoses, or treatment recommendations for individuals.
    - Always maintain a professional, informative, and neutral tone.
    - Clarify that your responses are for informational purposes only and not a substitute for professional medical consultation.
    - Focus on explaining conditions, symptoms, and general treatment approaches rather than prescribing actions.
    """)

rag_user_prompt_template = """
Based on the provided medical context from the Merck Manuals, please answer the following question thoroughly and concisely. Identify and explain relevant symptoms, causes, and recommended general treatments or solutions mentioned in the context. If the information is not explicitly found in the provided context, please state that you cannot find the answer in the given information. Remember, your response is for informational purposes only and should not be considered personalized medical advice.

Context: {context}
Question: {question}

"""

### Response Function

In [35]:
def generate_rag_response(user_query: str, k=3, max_tokens=600, temperature=0, top_p=0.95):
    global rag_system_prompt, rag_user_prompt_template

    # Retrieve general information
    context_documents = retriever.get_relevant_documents(query=user_query, k=k)
    context_list = [d.page_content for d in context_documents]

    # Combine documents chunks into a sigle context
    context_for_query = '. '.join(context_list)

    user_message = rag_user_prompt_template.format(context=context_for_query, question=user_query)

    # Generate respond

    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": rag_system_prompt},
                {"role": "user", "content": user_message}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p
        )
        # Extract and print the generate prompt
        response = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error Found:\n {e}")
        return None

    return response, context_for_query



## Question Answering using RAG

### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [36]:
response_with_rag_1, q1_context  = generate_rag_response(q1)
print(response_with_rag_1)


/tmp/ipython-input-145149224.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context_documents = retriever.get_relevant_documents(query=user_query, k=k)


The management of sepsis in a critical care unit involves several key steps and protocols as outlined in the provided context from the Merck Manuals. Here is a summary of the main components of sepsis management:

1. **Early Identification and Empiric Antibiotic Therapy**: Immediate administration of parenteral antibiotics after obtaining specimens for Gram stain and culture is crucial. This is because very prompt empiric therapy, started immediately after suspecting sepsis, can be lifesaving. The choice of antibiotics should be based on the suspected source of infection, clinical setting, knowledge or suspicion of causative organisms, sensitivity patterns common to the specific inpatient unit, and previous culture results. For septic shock of unknown cause, a regimen might include gentamicin or tobramycin combined with a 3rd-generation cephalosporin, or alternatives like ceftazidime plus a fluoroquinolone.

2. **Adjustment Based on Culture Results**: Once culture and sensitivity resul

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [37]:
response_with_rag_2, q2_context  = generate_rag_response(q2)
print(response_with_rag_2)

Appendicitis typically presents with several common symptoms. Initially, patients may experience epigastric or periumbilical pain, which after a few hours shifts to the right lower quadrant of the abdomen. This pain often intensifies with coughing or motion. Accompanying symptoms can include brief nausea, vomiting, and anorexia. Classic signs include direct and rebound tenderness at McBurney's point, Rovsing sign, psoas sign, and obturator sign. A low-grade fever is also commonly observed.

Appendicitis is primarily a surgical condition and cannot be effectively cured with medications alone. The standard treatment for acute appendicitis is an appendectomy, which can be performed using open or laparoscopic techniques. Preoperative management typically includes intravenous fluids and antibiotics, particularly third-generation cephalosporins. If the appendix has not perforated, no further antibiotics are generally required post-surgery. However, if there is perforation, antibiotics may be

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [38]:
response_with_rag_3, q3_context  = generate_rag_response(q3)
print(response_with_rag_3)

Sudden patchy hair loss, commonly seen as localized bald spots on the scalp, is typically referred to as alopecia areata. According to the Merck Manuals, alopecia areata is an autoimmune disorder that affects genetically susceptible individuals and may be triggered by environmental factors that are not clearly understood. This condition can result in hair loss on the scalp, beard, and potentially other hairy areas of the body, and in severe cases, it can progress to alopecia universalis, where most or all body hair is lost.

**Possible Causes:**
1. **Autoimmune Reaction:** The body's immune system mistakenly attacks the hair follicles, leading to hair loss.
2. **Genetic Susceptibility:** Individuals with a family history of alopecia areata or other autoimmune conditions are at higher risk.
3. **Environmental Triggers:** While not clearly defined, these could include stress, certain medications, or other external factors.

**Effective Treatments:**
1. **Corticosteroids:** These can be a

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [39]:
response_with_rag_4, q4_context  = generate_rag_response(q4)
print(response_with_rag_4)

For a person who has sustained a traumatic brain injury (TBI), resulting in temporary or permanent impairment of brain function, the recommended treatments according to the Merck Manuals include:

1. **Initial Treatment:**
   - **Ensuring a reliable airway** and maintaining adequate **ventilation and oxygenation**.
   - Maintaining **adequate blood pressure** to support brain perfusion.
   - **Surgery** may be necessary for severe cases to place monitors for tracking and treating intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.

2. **Post-Initial Treatment:**
   - In the days following the injury, it is crucial to continue maintaining adequate brain perfusion and oxygenation.
   - Preventing complications related to altered sensorium (changes in mental status) is also important.

3. **Rehabilitation:**
   - Many patients require rehabilitation, which can be extensive and continue for months or years, especially for tho

In [40]:
responses["Response with RAG"] = [response_with_rag_1, response_with_rag_2, response_with_rag_3, response_with_rag_4]
responses["Question Context"] = [q1_context, q2_context, q3_context, q4_context]
responses

,Question,Base Prompt Response,Response with Prompt Engineering,Response with RAG,Question Context
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a life-threatening condition trigger...,The management of sepsis in a critical care un...,Parenteral antibiotics should be given after s...
1,"What are the common symptoms for appendicitis,...",Appendicitis is an inflammation of the appendi...,Appendicitis is an inflammation of the appendi...,Appendicitis typically presents with several c...,Etiology\nAppendicitis is thought to result fr...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, which typically appea...","Sudden patchy hair loss, commonly seen as loca...","squaric acid dibutylester), or psoralen plus u..."
3,What treatments are recommended for a person w...,"Treating a physical injury to brain tissue, co...","Treatment for brain injuries, particularly tho...",For a person who has sustained a traumatic bra...,Chapter 324. Traumatic Brain Injury\nIntroduct...


# **Definig the LLM-as-a-Judge Evaluation function**

### Output Evaluation

### Prompt for Evaluation

In [41]:

groundedness_rater_system_messagedds_test = """
You are an expert at evaluating the factual groundedness of a generated answer with respect to a given context. Your task is to determine if every fact in the 'Answer' is directly supported by the information presented in the 'Context'.

Follow these strict rules:
1.  **Fact-checking**: Scrutinize each factual statement in the 'Answer'.
2.  **Context-only**: If a fact in the 'Answer' cannot be directly and explicitly found or inferred from the 'Context', then the Answer is NOT grounded.
3.  **No External Knowledge**: Do NOT use any external knowledge. Base your judgment SOLELY on the provided 'Context'.
4.  **Output Format**: Provide your assessment as a single word: 'Grounded' if all facts are supported by the context, or 'Not Grounded' if even one fact is not supported by the context.

Example:
Context: 'The Eiffel Tower is in Paris. It was built by Gustave Eiffel.'
Answer: 'The Eiffel Tower is in Paris and is a famous landmark.'
Result: Groundedd

Context: 'The Eiffel Tower is in Paris. It was built by Gustave Eiffel.'
Answer: 'The Eiffel Tower is in Rome and is a famous landmark.'
Result: Not Grounded

Context: 'The Eiffel Tower is in Paris.'
Answer: 'The Eiffel Tower is in Paris and was designed by Gustave Eiffel.'
Result: Not Grounded (because 'designed by Gustave Eiffel' is not explicitly in the context given)
"""




In [42]:
relevance_rater_system_message_test = """
You are an expert at evaluating the relevance of a generated 'Answer' to a given 'Question'. Your task is to determine if the 'Answer' directly and comprehensively addresses all aspects of the 'Question'.

Follow these strict rules:
1.  **Directness**: The 'Answer' must directly respond to the 'Question'. Avoid considering any information in the answer that does not pertain to the question.
2.  **Completeness**: If the 'Question' asks for multiple pieces of information, the 'Answer' should ideally provide all of them to be considered highly relevant.
3.  **No Extraneous Information**: The 'Answer' should not include significant information that is unrelated to the 'Question'.
4.  **Output Format**: Provide your assessment as a single word: 'Relevant' if the answer directly and thoroughly addresses the question, or 'Not Relevant' if it misses key aspects, contains significant irrelevant information, or does not answer the question directly.

Example:
Question: 'What are the common symptoms for appendicitis?'
Answer: 'Appendicitis commonly presents with abdominal pain, loss of appetite, nausea, and vomiting.'
Result: Relevant

Question: 'What are the common symptoms for appendicitis and its primary treatment?'
Answer: 'Appendicitis commonly presents with abdominal pain, loss of appetite, nausea, and vomiting.'
Result: Not Relevant (as it misses the treatment aspect)

Question: 'What are the common symptoms for appendicitis?'
Answer: 'The primary treatment for appendicitis is surgery. The appendix is a small, finger-like organ.'
Result: Not Relevant (as it does not address the symptoms)
"""

In [43]:
# Prompt to evaluate how relevant the answer is to the original question
relevance_rater_system_message = """
You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

# Prompt to evaluate how well the answer is grounded in the provided context
groundedness_rater_system_message = """
You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

In [44]:
user_message_template = """
    ###Question
    {question}

    ###Context
    {context}

    ###Answer
    {answer}
    """

## Google Gemini as Judge

In [45]:
def rate_ground_relevace_responses(question, response, context_for_query=""):
    filled_user_message_prompt = user_message_template.format(context=context_for_query, question=question, answer=response)

    gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.2
    )

    memory = ConversationBufferMemory(return_messages=True)

    messages_ground = [
                ("system", f"{groundedness_rater_system_message}"),
                ("human", f"{filled_user_message_prompt}")
            ]

    groundedness_response = gemini_llm.invoke(messages_ground).content

    messages_relevance = [
                ("system", f"{relevance_rater_system_message}"),
                ("human", f"{filled_user_message_prompt}")
            ]

    relevance_response = gemini_llm.invoke(messages_relevance).content

    return groundedness_response, relevance_response







#### **Evaluation 1: Base Prompt Response Evaluation**

In [49]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,base_prompt_response_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
**Rating: 2**

**Rationale:**

The answer provides a general, modern protocol for sepsis management, referencing the Surviving Sepsis Campaign (SSC) and a 2023 update. While the topic is the same as the context, the answer derives very little specific information from the provided text.

- The answer mentions early identification, specific symptoms (fever, tachycardia), lab tests (lactate), and fluid resuscitation (30 mL/kg of crystalloid). None of this information is present in the context.
- The context provides specific details on antibiotic regimens (e.g., gentamicin + cephalosporin), source control (draining abscesses), blood glucose management, and corticosteroid use, none of which are mentioned in the provided portion of the answer.
- The only significant overlap is the general principle of administering broad-spectrum antibiotics promptly after taking cultures. However, the answer frames this within the SSC's "one-hour" guideline, which is external informatio

#### **Evaluation 2: Prompt Engineering Response Evaluation**

In [50]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,response_with_prompt_eng_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
Rating: 2

Rationale: The answer provides a general overview of sepsis management, but several key points are not derived from the provided context. While the context does mention immediate antibiotic therapy (Answer point 2) and source control (Answer point 5), it does not mention early identification signs, fluid resuscitation, or supportive care like oxygen and blood pressure medications. These are major components of the AI's answer that are drawn from general medical knowledge, not the specific text provided.


******************************


Relevance Rate:
**Rating: 2**

**Rationale:**
The answer provides a very general overview of sepsis management, but it fails to draw upon the specific, detailed protocols listed in the provided context. The context gives explicit information on antibiotic regimens (e.g., gentamicin, cephalosporins, vancomycin), blood glucose management with insulin, corticosteroid therapy, and source control like draining abscesses. The an

#### **Evaluation 3: RAG Response Evaluation**

In [51]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,response_with_rag_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
Rating: 5

Rationale: The answer is a comprehensive and well-structured summary of the information provided in the context. Each point in the answer, from early antibiotic therapy and source control to supportive care and emerging therapies, is directly supported by specific details and statements found in the text. The AI has effectively organized the information into a clear protocol format, but all the content originates from the provided source material.


******************************


Relevance Rate:
**Rating:** 5

**Rationale:** The answer is fully relevant to the question. It comprehensively and accurately summarizes the protocol for managing sepsis as described in the provided context. It logically structures the information into key steps, including early antibiotic therapy, source control, supportive care (glucose management, corticosteroids), and emerging therapies, all of which are detailed in the source text.


## Actionable Insights and Business Recommendations

- During this project I Develop and evaluate a RAG-based AI solution using renowned medical manuals (provided as "medical_diagnosis_manual.pdf") to address healthcare challenges. The evaluation was focus on various rag metrics and involve integrating ChatGoogleGenerativeAI into the RAG pipeline as Judge.
- High Scores of **Groundedness and Relevance** I was able to get, Implementng RAG technology indicates the solution is effectevly leveraging the provided context to answer questions accurately and comprehensively

- Low Scores of **Groundedness and Relevance** I was able to get implementing Base Prompt and Prompt Engineering highligth areas where the LLM struglees without an explicit context aid or proper guidance these become opportuninites to improve , perhaps implementing different technies of prompt engineering or different LLM or Embedding Model
- RRAG model (Response with RAG) significantly outperforms both the base prompt and prompt-engineered responses in terms of groundedness and relevance. This is a crucial insight: RAG effectively uses the external knowledge base.

- Importance of Context: The poorer performance of the non-RAG models shows that without specific, relevant context, even advanced LLMs can hallucinate or provide generic answers that lack specific medical authority.
Value of Prompt Engineering: Even though it didn't match RAG, prompt engineering improved the initial general answers, showing the importance of guiding the LLM's behavior.

- Value of Prompt Engineering: Even though it didn't match RAG, prompt engineering can improve the initial general answers, showing the importance of guiding the LLM's behavior.


- Implementing a RAG architecture dramatically enhances the accuracy and context-specificity of medical information retrieval, directly addressing the challenge of information overload.
- Relying solely on general LLM knowledge can lead to less grounded and relevant medical responses, potentially risking misinformation.
- The quality of the provided context (Merck Manuals) is vital for ensuring high-quality, authoritative responses from the RAG system.

- Prioritize the deployment of the RAG-based AI solution in healthcare settings to provide medical professionals with highly accurate, context-specific information derived from trusted sources like the Merck Manuals. This will streamline decision-making and reduce the risk of information overload.
- Invest in expanding and maintaining the quality of the medical knowledge base (vector database). Regularly update the ingested manuals and documents to ensure the RAG system always provides the most current and comprehensive medical information.

- Develop training programs for healthcare staff on how to effectively use the RAG solution for quick access to critical information, and integrate the solution into existing clinical workflows to maximize its utility and impact on patient outcomes.

- Establish a feedback loop for continuous monitoring of the RAG system's performance, allowing for ongoing adjustments and improvements to the prompts, retrieval mechanisms, and knowledge base based on user experience and emerging medical knowledge.


<font size=6 color='#4682B4'>Power Ahead</font>
___